# Análise de Sentimento em reviews de Carros

Analise de sentimentos em comentários feitos sobre carros no site http://www.carrosnaweb.com.br/

Esses dados foram minerados utilizando a linguagem R e foram os dados da <b>Hackathon de Data Science do Laboratório de Estatística e Geoinformação da UFPR</b>

Agradeço ao pessoal da UFPR pela iniciativa, aos professores Walmes e Wagner.

O objetivo desse notebook é aplicar a técnica de analise de sentimentos utilizando-se de redes neurais recorrentes utilizando células de memória LSTM, para predição de sentimento em cima dos comentários de Opinião Geral fornecidos no Dataset.

## Estruturação dos Dados

Foi nos fornecidos 2 datasets distintos, "notas.csv" e "opinioes.json", notas.csv fornecia dados estruturados com notas para cada categoria em um carro (Interior,Acabamento, Estilo...), enquanto "opinioes.json" fornecia dados não estruturados contendo comentários reais feitos por usuários acerca dos carros revisados.

Os dados estruturados (notas para os quesitos) estão em arquivo CSV com campos delimitados por ; (notas.csv). As notas de um mesmo avaliador estão dispostas verticalmente, um quesito após o outro, ocupando assim 15 linhas no total. A ID formada por 8 dígitos hexadecimais identifica unicamente cada avaliação.

Os dados da porção semi/não estruturada estão em JSON (opinioes.json). Cada avaliação é um array JSON de 10 elementos (supostamente, pois podem haver discrepâncias acidentais). O primeiro elemento de cada array é a ID que permite pareamento dos dados das duas porções. Os demais elementos do array contém a informação de fabricante, modelo, especificações, ano, dono, cidade, distância percorrida, texto do campo pró, contra, defeitos, data da avaliação, etc.

In [71]:
from string import punctuation
import tensorflow as tf
import pandas as pd
import numpy as np

In [72]:
notas = pd.read_csv("./notas.csv", sep = ';', encoding = "utf-8")

opinioes = pd.read_json("./opinioes.json", encoding = "utf-8")

In [73]:
notas.head(15)

,ID,quesito,nota
0,e2b9dc08,Estilo,7
1,e2b9dc08,Acabamento,5
2,e2b9dc08,Posição de dirigir,7
3,e2b9dc08,Instrumentos,10
4,e2b9dc08,Interior,5
5,e2b9dc08,Porta-malas,5
6,e2b9dc08,Desempenho,10
7,e2b9dc08,Motor,10
8,e2b9dc08,Câmbio,10
9,e2b9dc08,Freios,10


<i>O dataset Notas.csv fornece uma nota fornecida pelo usuário para cada feature do carro avaliado, será feita uma média dessas features, para servir como media final e label para a analise de sentimento.</i>

In [74]:
opinioes.drop([1,2,3,4,5,6,7,9], inplace = True, axis = 1)
columns = ['ID','Opiniao Geral']

opinioes.columns = columns
opinioes = opinioes.sort_values(by = ['ID'], axis = 0)

In [75]:
opinioes.head(10)

,ID,Opiniao Geral
5181,00022137,"Opinião Geral:É um bom carro, mas pelo valor d..."
4467,000a582d,Opinião Geral:pode melhorar
3945,001e058b,"Opinião Geral:Estou feliz com meu carrinho, só..."
2354,002f1155,Opinião Geral:Para quem quer um carro automáti...
2009,00372114,Opinião Geral:Em geral o carro é bom e chama a...
995,006c0f2c,"Opinião Geral:carro sem frescuras, e para quem..."
3868,00766a5c,"Opinião Geral:Quem tem um e conserva bem,tem c..."
1449,007684da,Opinião Geral:A relação custo-beneficio atende...
4805,0090acd0,Opinião Geral:Pessimo carro. Ainda irei adesiv...
4418,009d1244,Opinião Geral:Pra sair do gacho o carro atende...


No dataset de opinião, dropamos todas as colunas com exceção da coluna de ID e Opinião Geral, na qual iremos performar a analise de sentimentos, falta adicionar a média dada pelo usuario para o carro, porém essa média esta no dataset notas, o código abaixo realiza a média e adiciona a coluna média a Opinião Geral

In [76]:
def media_geral():
    
    medias = []
    for unique_id in sorted(notas['ID'].unique()):
        
        media = np.average(notas.loc[notas.ID == unique_id]['nota'])
        #print(unique_id, media)
        medias.append(media)
    
    return medias

In [77]:
medias = media_geral()
opinioes['medias'] = pd.Series(medias).values
opinioes['Opiniao Geral'] = opinioes['Opiniao Geral'].apply(lambda x: x[14:].lower())

In [78]:
opinioes.head(10)

,ID,Opiniao Geral,medias
5181,00022137,"é um bom carro, mas pelo valor dele novo, comp...",7.133333
4467,000a582d,pode melhorar,7.733333
3945,001e058b,"estou feliz com meu carrinho, só me deu alegri...",8.000000
2354,002f1155,"para quem quer um carro automático de verdade,...",8.666667
2009,00372114,em geral o carro é bom e chama atenção por ond...,6.933333
995,006c0f2c,"carro sem frescuras, e para quem não quer fica...",8.600000
3868,00766a5c,"quem tem um e conserva bem,tem carro pra muito...",7.866667
1449,007684da,a relação custo-beneficio atende a expectativa...,8.000000
4805,0090acd0,pessimo carro. ainda irei adesivar no vidro tr...,4.466667
4418,009d1244,"pra sair do gacho o carro atende, porem o segu...",4.933333


Adicionado então a coluna de média ao dataset de opinioes, foi feita também uma trimagem nos comentarios, 
pois todos começavam com "Opinião Geral:", além disso retirou-se qualquer caps-lock dos comentarios, afim de reduzir-se ruidos no dataset

In [79]:
#Função que retira qualquer pontuação ddos comentários

def clean_punctuation(aux):
    
    for punct in punctuation:
        if punct in aux:
            aux = ''.join(aux).split(punct)
            aux = ' '.join(aux)
    
    return aux

In [80]:
opinioes['Opiniao Geral'] = opinioes['Opiniao Geral'].apply(lambda x: clean_punctuation(x))

In [81]:
opinioes.head(50)

,ID,Opiniao Geral,medias
5181,00022137,é um bom carro mas pelo valor dele novo comp...,7.133333
4467,000a582d,pode melhorar,7.733333
3945,001e058b,estou feliz com meu carrinho só me deu alegri...,8.000000
2354,002f1155,para quem quer um carro automático de verdade ...,8.666667
2009,00372114,em geral o carro é bom e chama atenção por ond...,6.933333
995,006c0f2c,carro sem frescuras e para quem não quer fica...,8.600000
3868,00766a5c,quem tem um e conserva bem tem carro pra muito...,7.866667
1449,007684da,a relação custo beneficio atende a expectativa...,8.000000
4805,0090acd0,pessimo carro ainda irei adesivar no vidro tr...,4.466667
4418,009d1244,pra sair do gacho o carro atende porem o segu...,4.933333


### Encoding das Labels

Como será um classificador binário de sentimento (positivo ou negativo), qualquer comentario cuja média geral dada pelo usuário for maior que 6, será considerado um comentário positivo, caso contrario, será um comentário negativo.

In [82]:
threshold = 6
opinioes['medias'] = opinioes['medias'].apply(lambda x: 1 if x > threshold else 0)
opinioes.drop('ID', inplace = True, axis = 1)

### Quantidade de opiniões positivas e negativas

Percebe-se que que a grande maioria das opiniões são positivas (que possuem uma média geral maior que 6),
esse é um caso de um dataset desbalanceado, em que a quantidade de informação por classe não é igual,
isso vai acarretar em acurácias diferentes para predição de comentarios positivos e negativos.

Espera-se que a rede neural aprenderá com mais facilidade e possuira uma melhor acuracia em opiniões positivas
do que negativas.

Uma solução para esse problema seria minerar mais dados do site para enriquecer o dataset, porém isso não será feito nesse notebook.

Continuaremos com a analise e no final mostrarei a diferente acurácia para cada classe devidda a esse desbalanceamento

In [83]:
opinioes['medias'].value_counts()

1    4358
0     971
Name: medias, dtype: int64

Para as colunas de labels, se a media geral for maior que o threshold minimo (no caso foi escolhido 6) então a avaliação é positiva, caso contrario a avaliação é negativa.

In [84]:
'''
cria uma lista com as palavras dos reviews
'''
words = []
for index,row in opinioes['Opiniao Geral'].iteritems():
    for word in row.split():
        words.append(word)

In [85]:
'''
para cada palavra do vocabulario atribui-se um valor numerico
'''
vocab_to_int = dict()
counter = 1
for word in words:
    if word not in vocab_to_int.keys():
        vocab_to_int[word] = counter
        counter+=1
    else:
        pass

In [86]:
review_ints = []    
for index,review in opinioes['Opiniao Geral'].iteritems():
    review_ints.append([vocab_to_int[word] for word in review.split()])
    
labels = opinioes['medias'].tolist()

É necessario fazer a conversão de palavras para inteiros, para isso foi criado um bag de palavras na variavel words, pegando todas as palavras dos reviews, então é criado um dicionario, atribuindo um vlaor inteiro unico para cada palavra.Então cada review em Opinioes é traduzido para inteiros e armazenado na variavel <b>review_ints</b>

In [87]:
from collections import Counter

review_lens = Counter([len(x) for x in review_ints])
print("Reviews com tamanho zero: {}".format(review_lens[0]))
print("Review mais longo: {}".format(max(review_lens)))

Reviews com tamanho zero: 16
Review mais longo: 443


Agora, fazendo uma analise do tamanho dos comentarios, percebe-se a existencia de comentários vazios ou comentários muito longos, os comentarios com zeros palavras serão removidos e o restante truncado em 200 palavras.

In [88]:
non_zero_idx = [ii for ii, review in enumerate(review_ints) if len(review) != 0]
len(non_zero_idx)

review_ints = [review_ints[ii] for ii in non_zero_idx]
labels = np.array([labels[ii] for ii in non_zero_idx])

seq_len = 200
features = np.zeros((len(review_ints), seq_len), dtype=int)
for i, row in enumerate(review_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [89]:
features.shape

(5313, 200)

Restaram então 5313 comentários, que foram truncados em um tamanho fixo de 200.
Caso o comentario tenha menos de 200 palavras, os espaços vazios serão preenchidos com 0's.

O array features, fiocu da seguinte forma então:


In [90]:
features[0:10]

array([[  0,   0,   0, ...,  29,  30,  31],
       [  0,   0,   0, ...,   0,  32,  33],
       [  0,   0,   0, ...,  65,  66,  67],
       ..., 
       [  0,   0,   0, ..., 162, 156, 163],
       [  0,   0,   0, ..., 174,   2,  60],
       [  0,   0,   0, ..., 182,  21, 183]])

## Treinamento, Validação, Teste

Com os dados organizados, é necessário fazer um split do dataset pra treinamento,validação e teste
A proporção ficou em 80% para treinamento 10% para validação e 10% para testes. Podendo ser alterado essa proporção na variavel train_pct

In [91]:
train_pct = 0.8

split_idx = int(len(features)*train_pct)

train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shape:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shape:
Train set: 		(4250, 200) 
Validation set: 	(531, 200) 
Test set: 		(532, 200)


## Construindo o Grafo da Rede Neural

In [92]:
'''
lstm_size -> tamanho das layers de LSTM
lstm_layers -> quantidade de layers, começamos em 1 e aumentamos apara analisar melhorias
batch_size -> quantidade de reviews inputadas na rede, varia de acordo com a quantidade de memória 
disponível na maquina

learning_rate -> taxa de aprendizado
'''

lstm_size = 512
lstm_layers = 1
batch_size = 256
learning_rate = 0.001

In [93]:
n_words = len(vocab_to_int) + 1 

#Grafo da rede
graph = tf.Graph()

with graph.as_default():
    inputs_ = tf.placeholder(tf.int32,[None,None], name = 'inputs') 
    labels_ = tf.placeholder(tf.int32, [None,None], name = 'labels')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')

In [94]:
embed_size = 300

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [95]:
with graph.as_default():
    
    #Node de LSTM 
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    #Dropout
    drop = tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob = keep_prob)
    
    #Stack de multiplas camadas LSTM
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    #Estado inicial em 0
    initial_state = cell.zero_state(batch_size, tf.float32)

In [96]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                   initial_state=initial_state)

In [97]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn = tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [98]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [99]:
#Função para retornar batches de dados

def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [100]:
epochs = 15

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

Epoch: 0/15 Iteration: 5 Train loss: 0.162
Epoch: 0/15 Iteration: 10 Train loss: 0.150
Epoch: 0/15 Iteration: 15 Train loss: 0.134
Epoch: 1/15 Iteration: 20 Train loss: 0.139
Epoch: 1/15 Iteration: 25 Train loss: 0.137
Val acc: 0.814
Epoch: 1/15 Iteration: 30 Train loss: 0.108
Epoch: 2/15 Iteration: 35 Train loss: 0.126
Epoch: 2/15 Iteration: 40 Train loss: 0.108
Epoch: 2/15 Iteration: 45 Train loss: 0.121
Epoch: 3/15 Iteration: 50 Train loss: 0.076
Val acc: 0.771
Epoch: 3/15 Iteration: 55 Train loss: 0.107
Epoch: 3/15 Iteration: 60 Train loss: 0.077
Epoch: 4/15 Iteration: 65 Train loss: 0.071
Epoch: 4/15 Iteration: 70 Train loss: 0.068
Epoch: 4/15 Iteration: 75 Train loss: 0.065
Val acc: 0.793
Epoch: 4/15 Iteration: 80 Train loss: 0.078
Epoch: 5/15 Iteration: 85 Train loss: 0.060
Epoch: 5/15 Iteration: 90 Train loss: 0.055
Epoch: 5/15 Iteration: 95 Train loss: 0.076
Epoch: 6/15 Iteration: 100 Train loss: 0.058
Val acc: 0.832
Epoch: 6/15 Iteration: 105 Train loss: 0.049
Epoch: 6/15 Ite

In [101]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Acurácia média: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints/sentiment.ckpt
Acurácia média: 0.727


In [102]:
print("Acurácia por classe (1) (0):\n\n", test_acc)

Acurácia por classe (1) (0):

 [0.75, 0.703125]


### Resultados

Perecebe-se que a classificação de comentários obteve uma melhor acuracia em comentarios positivos, conforme esperado, devido a quantidade desbalanceada de comentários por classe. Ainda assim a diferença não foi tão grande quando esperava-se.

Esse tipo de processamento de sentimentos, pode ser utilizado para monitorar indices de satisfação em comentários da páginas de Facebook, Fórums, Comentários em site de Notícias e etc, a aplicabilidade do conceito é vasta.

Com esse dataset foi somente mostrado uma prova de conceito do uso de RNN utilizando células LSTM, que contrário as redes feedfoward clássicas, que aprendem palavra por palavra,essas,conseguem estabelecer relações semanticas entre palavras numa mesma frase, gerando resultados mais confiáveis.

Outro ponto importante a ressaltar é a acurácia por quantidade de dados, a quantidade de comentários utilizada nesse notebook foi relativamente pequena, com um dataset mais robusto, a qualidade e assertividade do modelo tende a aumentar.